In [1]:
import nltk
import os
import re
import math
import operator
import json
import tqdm

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ekansh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/ekansh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ekansh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
       lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words
# def stem_words(words):
#     stemmed_words = []
#     for word in words:
#        stemmed_words.append(stemmer.stem(word))
    return stemmed_words
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text
def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb
def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf
def tf_idf_score(tf,idf):
    return tf*idf
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf
def sentence_importance(sentence,dict_freq,sentences):
     sentence_score = 0
     sentence = remove_special_characters(str(sentence)) 
     sentence = re.sub(r'\d+', '', sentence)
     pos_tagged_sentence = [] 
     no_of_sentences = len(sentences)
     pos_tagged_sentence = pos_tagging(sentence)
     for word in pos_tagged_sentence:
          if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
                word = word.lower()
                word = wordlemmatizer.lemmatize(word)
                sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
     return sentence_score


In [3]:
# writefile = open('summary.txt', 'w')       

def func (text):
    tokenized_sentence = sent_tokenize(text)
    text = remove_special_characters(str(text))
    text = re.sub(r'\d+', '', text)
    tokenized_words_with_stopwords = word_tokenize(text)
    tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
    tokenized_words = [word for word in tokenized_words if len(word) > 1]
    tokenized_words = [word.lower() for word in tokenized_words]
    tokenized_words = lemmatize_words(tokenized_words)
    word_freq = freq(tokenized_words)
    # input_user = int(input('Percentage of information to retain(in percent):'))
    # no_of_sentences = int((input_user * len(tokenized_sentence))/100)
    no_of_sentences = 1
    # print(no_of_sentences)
    c = 1
    sentence_with_importance = {}
    # print(len(tokenized_sentence))
    # print('\n')
    for sent in tokenized_sentence:
        # print(c)
        # print('\n')
        sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)
        sentence_with_importance[c] = sentenceimp
        c = c+1
    # print(c)
    sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)
    cnt = 0
    summary = []
    sentence_no = []
    # print(cnt)

    for word_prob in sentence_with_importance:
        if cnt < no_of_sentences:
            sentence_no.append(word_prob[0])
            cnt = cnt+1
        else:
            break
    sentence_no.sort()
    cnt = 1
    for sentence in tokenized_sentence:
        if cnt in sentence_no:
            summary.append(sentence)
        cnt = cnt+1
    summary = " ".join(summary)
    # print("\n")
    # print("Summary:")
    # print(summary)
    return summary


# read the json file
with open('testingpurpose.jsonl') as f:
    data = f.readlines()
    # print(data)

    finalarr = []

    for i in tqdm.tqdm(data):
        ni = json.loads(i)
        text = ni['article']
        stri = func(text)
        finalarr.append(stri)

  0%|          | 0/241 [00:00<?, ?it/s]

  0%|          | 1/241 [04:01<16:04:03, 241.02s/it]


KeyboardInterrupt: 

In [ ]:
print(finalarr)

['5×10–5 bar–1 ) .', 'For example , wrongly rejecting a food item might see an animal missing a meal , but wrongly accepting a food item could see an animal ingesting poison .', 'ac .', 'Hosts were only included in these analyses if they had >35 samples in either the juvenile and prime age class or prime age and older adult class ( no host had >35 samples in all three age classes; 13 hosts were included in the juvenile and matched prime age adult groups; a separate set of 13 hosts were included in the older adult and matched prime age adult groups ) .', 'doi .', '25 ng/µl .', 'ilmn12 .', ', chr17q21 locus ) .', 'Fertilizers were applied three times per year , namely once just before sowing ( 50 kg/ha N , 85 kg/ha P , 50 kg/ha K ) , once when wheat was at the tillering stage ( 50 kg/ha N , 85 kg/ha P , 50 kg/ha K ) , and once when wheat was flowering ( 20 kg/ha N , 34 kg/ha P , 20 kg/ha K ) .', 'Then two choice targets appeared at 1 of 3 different pairs of positions ( 140 mm to the left

In [ ]:
# write to a file with each summary in a new line
outF = open('summary.txt',"w")
for line in finalarr:
  print(line)
  outF.write(line)
  outF.write("\n")

5×10–5 bar–1 ) .
For example , wrongly rejecting a food item might see an animal missing a meal , but wrongly accepting a food item could see an animal ingesting poison .
ac .
Hosts were only included in these analyses if they had >35 samples in either the juvenile and prime age class or prime age and older adult class ( no host had >35 samples in all three age classes; 13 hosts were included in the juvenile and matched prime age adult groups; a separate set of 13 hosts were included in the older adult and matched prime age adult groups ) .
doi .
25 ng/µl .
ilmn12 .
, chr17q21 locus ) .
Fertilizers were applied three times per year , namely once just before sowing ( 50 kg/ha N , 85 kg/ha P , 50 kg/ha K ) , once when wheat was at the tillering stage ( 50 kg/ha N , 85 kg/ha P , 50 kg/ha K ) , and once when wheat was flowering ( 20 kg/ha N , 34 kg/ha P , 20 kg/ha K ) .
Then two choice targets appeared at 1 of 3 different pairs of positions ( 140 mm to the left and right of the central fix

In [ ]:
# write to a file with each summary in a new line
outF = open('summary.txt',"w")
for line in finalarr:
  print(line)
  outF.write(line)
  outF.write("\n")

5×10–5 bar–1 ) .
For example , wrongly rejecting a food item might see an animal missing a meal , but wrongly accepting a food item could see an animal ingesting poison .
ac .
Hosts were only included in these analyses if they had >35 samples in either the juvenile and prime age class or prime age and older adult class ( no host had >35 samples in all three age classes; 13 hosts were included in the juvenile and matched prime age adult groups; a separate set of 13 hosts were included in the older adult and matched prime age adult groups ) .
doi .
25 ng/µl .
ilmn12 .
, chr17q21 locus ) .
Fertilizers were applied three times per year , namely once just before sowing ( 50 kg/ha N , 85 kg/ha P , 50 kg/ha K ) , once when wheat was at the tillering stage ( 50 kg/ha N , 85 kg/ha P , 50 kg/ha K ) , and once when wheat was flowering ( 20 kg/ha N , 34 kg/ha P , 20 kg/ha K ) .
Then two choice targets appeared at 1 of 3 different pairs of positions ( 140 mm to the left and right of the central fix